In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Path to the main folder containing subfolders for each class
data_dir = "../Data"

# Create training and validation datasets
batch_size = 32
img_size = (256, 256)
seed = 123

# Load the data and split it into training and validation sets
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",  # This specifies the subset to be used for training
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale"  # Important: images are in grayscale, so specify this mode
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",  # This specifies the subset to be used for validation
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale"  # Same as for training, images are in grayscale
)


2025-04-10 13:05:27.301006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744311927.315296   29083 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744311927.319892   29083 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 13:05:27.336062: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 3096 files belonging to 4 classes.
Using 2477 files for training.


I0000 00:00:1744311929.779324   29083 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6


Found 3096 files belonging to 4 classes.
Using 619 files for validation.


In [2]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# Build the model using a sequential approach
model = tf.keras.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 1)),  # Normalize pixel values to [0, 1]

    # Convolutional Layer 1: 64 filters, 3x3 kernel, ReLU activation, and same padding
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(),  # MaxPooling to reduce spatial dimensions
    layers.Dropout(0.4),  # Dropout to prevent overfitting after the first convolutional layer

    # Convolutional Layer 2: 128 filters, 3x3 kernel, ReLU activation, and same padding
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Dropout(0.4),  # Dropout after the second convolutional layer

    # Convolutional Layer 3: 256 filters, 3x3 kernel, ReLU activation, and same padding
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Dropout(0.4),  # Dropout after the third convolutional layer

    layers.Flatten(),  # Flatten the output to feed into the fully connected layer
    layers.Dense(256, activation='relu'),  # Dense layer with 256 units and ReLU activation
    layers.Dropout(0.5),  # Dropout on the fully connected layer to reduce overfitting
    layers.Dense(4, activation='softmax')  # Output layer for multi-class classification (4 classes)
])

# Compile the model with Adam optimizer and sparse categorical crossentropy loss
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Suitable for multi-class classification with integer labels
    metrics=['accuracy']  # Track accuracy during training
)


/home/david/projects/tf217/env/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True  # Stop training if validation loss doesn't improve for 10 epochs, restore best weights
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6  # Reduce learning rate by a factor of 0.5 if validation loss plateaus for 3 epochs
)

# Training the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=59,  # Train for 59 epochs
    callbacks=[early_stop, lr_scheduler]  # Use early stopping and learning rate scheduler callbacks
)


Epoch 1/59


I0000 00:00:1744311933.714434   29166 service.cc:148] XLA service 0x7f25d0004ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744311933.714503   29166 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-04-10 13:05:33.745669: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744311933.881162   29166 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744311942.388247   29166 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


78/78 ━━━━━━━━━━━━━━━━━━━━ 24s 167ms/step - accuracy: 0.3237 - loss: 2.1482 - val_accuracy: 0.4701 - val_loss: 1.2742 - learning_rate: 0.0010
Epoch 2/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.5131 - loss: 1.1790 - val_accuracy: 0.5897 - val_loss: 1.0273 - learning_rate: 0.0010
Epoch 3/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.5834 - loss: 0.9881 - val_accuracy: 0.6947 - val_loss: 0.8716 - learning_rate: 0.0010
Epoch 4/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - accuracy: 0.6643 - loss: 0.8181 - val_accuracy: 0.7383 - val_loss: 0.7345 - learning_rate: 0.0010
Epoch 5/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.7190 - loss: 0.7168 - val_accuracy: 0.7577 - val_loss: 0.6793 - learning_rate: 0.0010
Epoch 6/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.7689 - loss: 0.6075 - val_accuracy: 0.7803 - val_loss: 0.6600 - learning_rate: 0.0010
Epoch 7/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.7967 - loss: 0.5101 - val_accuracy: 0.

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# Load the pre-trained Xception model without the final layers
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the Xception layers to prevent them from being updated during training
base_model.trainable = False

# Build the model by stacking layers
model_xception = models.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 1)),  # Normalize pixel values to [0, 1]
    layers.Conv2D(3, (3, 3), padding='same', activation='relu'),  # Convert grayscale images to 3 channels for Xception input
    base_model,  # Use the pre-trained Xception model as a base
    layers.GlobalAveragePooling2D(),  # Global average pooling to reduce spatial dimensions
    layers.Dense(256, activation='relu'),  # Dense layer with 256 units and ReLU activation
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(4, activation='softmax')  # Output layer for multi-class classification (4 classes)
])

# Compile the model with Adam optimizer and sparse categorical crossentropy loss
model_xception.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/home/david/projects/tf217/env/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Training the model_2 (Xception + custom layers)
history_xception = model_xception.fit(
    train_ds,
    validation_data=val_ds,
    epochs=59,  # Number of epochs for training
    callbacks=[early_stop, lr_scheduler],  # Callbacks for EarlyStopping and ReduceLROnPlateau
)


Epoch 1/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 36s 283ms/step - accuracy: 0.4874 - loss: 1.1352 - val_accuracy: 0.7528 - val_loss: 0.6917 - learning_rate: 0.0010
Epoch 2/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - accuracy: 0.7327 - loss: 0.6741 - val_accuracy: 0.7932 - val_loss: 0.5757 - learning_rate: 0.0010
Epoch 3/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - accuracy: 0.7812 - loss: 0.5714 - val_accuracy: 0.7803 - val_loss: 0.5680 - learning_rate: 0.0010
Epoch 4/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - accuracy: 0.7900 - loss: 0.5303 - val_accuracy: 0.7997 - val_loss: 0.5327 - learning_rate: 0.0010
Epoch 5/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - accuracy: 0.8322 - loss: 0.4812 - val_accuracy: 0.8045 - val_loss: 0.5136 - learning_rate: 0.0010
Epoch 6/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 120ms/step - accuracy: 0.8513 - loss: 0.4103 - val_accuracy: 0.8174 - val_loss: 0.5216 - learning_rate: 0.0010
Epoch 7/59
78/78 ━━━━━━━━━━━━━━━━━━━━ 10s 126ms/step - accuracy: 0.8548 - loss: 0.4082 -

In [24]:
# Evaluate both models on the validation dataset
loss_original, accuracy_original = model.evaluate(val_ds)
loss_xception, accuracy_xception = model_xception.evaluate(val_ds)

# Print the comparison of the two models
print(f"Original Model - Loss: {loss_original:.4f}, Accuracy: {accuracy_original:.4f}")
print(f"Xception Model - Loss: {loss_xception:.4f}, Accuracy: {accuracy_xception:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8833 - loss: 0.4616
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.8728 - loss: 0.4042
Original Model - Loss: 0.4902, Accuracy: 0.8821
Xception Model - Loss: 0.4256, Accuracy: 0.8691
